# setup

In [1]:
import torch
import os
import matplotlib.pyplot as plt

from utils import Imagenet_xy, train, eval, model_select, plot_log

from tqdm import tqdm
from collections import OrderedDict

In [2]:
BATCH_SIZE = 128
EPOCHS = 1

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
if not os.path.exists('output'):
    os.makedirs('output')

# data

In [4]:
trn = Imagenet_xy('train')
val = Imagenet_xy('val')

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/39 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

In [5]:
trn_loader = torch.utils.data.DataLoader(trn, batch_size = BATCH_SIZE, shuffle = True)
val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE)

trn.__len__() / BATCH_SIZE, val.__len__() / BATCH_SIZE

390.625

# model

In [6]:
origem = 'rn18'
espelhado = 'rn34'
model_label = origem + espelhado
backbone = torch.load('output/{}.pt'.format(model_label))

model = model_select(origem, pretrained = False)

Using cache found in /home/josegfer/.cache/torch/hub/pytorch_vision_v0.10.0


In [7]:
key_transformation = []
for key in model.state_dict().keys():
    key_transformation.append(key)

In [8]:
# from https://gist.github.com/the-bass/0bf8aaa302f9ba0d26798b11e4dd73e3

state_dict = backbone.state_dict()
new_state_dict = OrderedDict()

for i, (key, value) in enumerate(state_dict.items()):
    new_key = key_transformation[i]
    new_state_dict[new_key] = value

In [9]:
# log = model.load_state_dict(backbone.state_dict(), strict = False)
log = model.load_state_dict(new_state_dict, strict = False)
assert log.missing_keys == ['fc.weight', 'fc.bias']

In [7]:
model = model.to(device)

# train

In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [9]:
log = []
for epoch in (range(EPOCHS)):
    loss_trn, log_trn = train(model, val_loader, optimizer, criterion, device, log = True)
    loss_val = eval(model, val_loader, criterion, device)

    log.append([loss_trn, loss_val])
    plot_log(log_trn, loss_val = loss_val, epoch = epoch)
    torch.save(model.cpu(), 'output/partial_{}_finetuned.pt'.format(model_label))

  0%|          | 0/391 [00:00<?, ?it/s]/home/josegfer/miniconda3/envs/mirror/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 391/391 [03:25<00:00,  1.90it/s]


In [11]:
print(log)

[[9.40934645123494, 7.161983783897536]]


In [12]:
# torch.save(model.cpu(), 'output/{}_finetuned.pt'.format(model_label))